<a href="https://colab.research.google.com/github/wirt30435/travel_product_matching/blob/main/%E6%AF%94%E5%B0%8D%E6%97%85%E9%81%8A%E7%94%A2%E5%93%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
# 📌 安裝必要套件（只需執行一次）
!pip install --upgrade google-cloud-bigquery
!pip install pandas jieba rapidfuzz

In [7]:
# 📌 安裝套件
!pip install --upgrade google-cloud-bigquery
!pip install jieba rapidfuzz

# 📌 匯入套件
import pandas as pd
import jieba
from google.colab import auth
from google.cloud import bigquery
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from datetime import datetime
import itertools
import os

# ✅ Step 1: Google 認證
auth.authenticate_user()

# ✅ Step 2: BigQuery client 初始化
project_id = "bishare-1606"
dataset = "jessica_test"
client = bigquery.Client(project=project_id)

# ✅ Step 3: 載入 A/B 表資料
vendor_A_table = f"{project_id}.{dataset}.vendor_A"
vendor_B_table = f"{project_id}.{dataset}.vendor_B"

query_A = f"SELECT product_id, product_name, product_location_country, price FROM `{vendor_A_table}`"
query_B = f"SELECT product_id, product_name, product_location_country, price FROM `{vendor_B_table}`"

df_A = client.query(query_A).to_dataframe()
df_B = client.query(query_B).to_dataframe()

# ✅ Step 4: 中文斷詞 function
def tokenize(text):
    return set(jieba.lcut(text))

# ✅ Step 5: 比對 function（for multiprocessing）
def compare_product(row_a):
    row_results = []
    tokens_a = tokenize(row_a['product_name'])
    country = row_a['product_location_country']

    df_b_filtered = df_B[df_B['product_location_country'] == country]

    best_score = 0
    best_b_row = None

    for _, row_b in df_b_filtered.iterrows():
        tokens_b = tokenize(row_b['product_name'])

        if abs(len(tokens_a) - len(tokens_b)) > 5:
            continue

        intersection = tokens_a & tokens_b
        union = tokens_a | tokens_b
        jaccard_score = len(intersection) / len(union) if union else 0

        if jaccard_score > best_score:
            best_score = jaccard_score
            best_b_row = row_b

    if best_b_row is not None:
        vendor_a_price = row_a['price'] if row_a['price'] is not None else 0
        vendor_b_price = best_b_row['price'] if best_b_row['price'] is not None else 0
        price_diff = vendor_b_price - vendor_a_price

        row_results.append({
            'product_location_country': country,
            'vendor_A_product_id': row_a['product_id'],
            'vendor_A_product_name': row_a['product_name'],
            'vendor_A_price': vendor_a_price,
            'vendor_B_product_id': best_b_row['product_id'],
            'vendor_B_product_name': best_b_row['product_name'],
            'vendor_B_price': vendor_b_price,
            'jaccard_score': best_score,
            'price_diff': price_diff
        })

    return row_results


# ✅ Step 6: 多進程處理比對
print("🚀 啟動多核心比對中...")

with Pool(processes=os.cpu_count()) as pool:
    all_results = list(tqdm(pool.imap(compare_product, df_A.to_dict(orient='records')), total=len(df_A)))

# ✅ Step 7: 組合結果
flat_results = list(itertools.chain.from_iterable(all_results))
matched_df = pd.DataFrame(flat_results)

# ✅ Step 8: 寫入 BigQuery（如果不存在會自動建立）
today_str = datetime.today().strftime('%Y%m%d')
destination_table = f"{dataset}.experience_products_matched_results_{today_str}"

matched_df.to_gbq(destination_table=destination_table,
                  project_id=project_id,
                  if_exists='replace')  # 可改成 append

print("✅ 比對完成並已寫入 BigQuery！")



🚀 啟動多核心比對中...


100%|██████████| 3473/3473 [17:49<00:00,  3.25it/s]
/tmp/ipython-input-7-1950077303.py:97: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  matched_df.to_gbq(destination_table=destination_table,
100%|██████████| 1/1 [00:00<00:00, 2356.35it/s]

✅ 比對完成並已寫入 BigQuery！
